In [9]:
import os
from os.path import join
import pandas as pd

task = "covert"   
# Define directories
wd = r'C:\Users\Radovan\OneDrive\Radboud\Studentships\Jordy Thielen\root'
os.chdir(wd)
data_dir = join(wd, "data")
experiment_dir = join(data_dir, "experiment")
files_dir = join(experiment_dir, 'files')
sourcedata_dir = join(experiment_dir, 'sourcedata')
derivatives_dir = join(join(experiment_dir, 'derivatives'))
ica_dir = join(data_dir, 'analysis', 'ICA')

In [10]:
import os
import pandas as pd
from os.path import join

# Base directory for subject folders
base_dir = join(ica_dir, "ica")

# Lists and dictionaries to accumulate data per subject
consolidated_data = []         # for writing the CSV summary
subjects_channel_reject = {}   # for writing the dict mapping subject : rejected channels

# Loop through each subject directory in the base folder
for subject_dir in os.listdir(base_dir):
    subject_path = os.path.join(base_dir, subject_dir)
    
    # Make sure it is a directory (each representing a subject)
    if not os.path.isdir(subject_path):
        continue
    
    # Define subject ID (modify if needed to remove any prefixes)
    subject_id = subject_dir

    # Define file names for the three CSV files
    bad_elec_file     = os.path.join(subject_path, f"{subject_dir}_bad_elec.csv")
    deleted_comp_file = os.path.join(subject_path, f"{subject_dir}_deleted_eye_components.csv")
    retained_comp_file= os.path.join(subject_path, f"{subject_dir}_retained_noise_components.csv")
    
    # Initialize lists for channels/components and their reasons
    rej_channels, rej_channels_reason = [], []
    rej_components, rej_components_reason = [], []
    retained_components, retained_components_reason = [], []
    
    # 1) Read the "bad_elec.csv" for rejected channels
    if os.path.exists(bad_elec_file):
        df_bad = pd.read_csv(bad_elec_file)
        # Assumes columns "Electrode" and "Reason"; adjust if your CSV uses different column names.
        rej_channels = df_bad["Electrode"].astype(str).tolist()
        rej_channels_reason = df_bad["Reason"].astype(str).tolist()
    else:
        print(f"File not found: {bad_elec_file}")
    
    # Save the rejected channels list into the dictionary (for the txt output)
    subjects_channel_reject[subject_id] = rej_channels
    
    # 2) Read the "deleted_eye_components.csv" for rejected components
    if os.path.exists(deleted_comp_file):
        df_del = pd.read_csv(deleted_comp_file)
        # Assumes columns "Component" and "Reason"
        rej_components = df_del["Component"].astype(str).tolist()
        rej_components_reason = df_del["Reason"].astype(str).tolist()
    else:
        print(f"File not found: {deleted_comp_file}")
    
    # 3) Read the "retained_noise_components.csv" for retained components
    if os.path.exists(retained_comp_file):
        df_ret = pd.read_csv(retained_comp_file)
        # Assumes columns "Component" and "Reason"
        retained_components = df_ret["Component"].astype(str).tolist()
        retained_components_reason = df_ret["Reason"].astype(str).tolist()
    else:
        print(f"File not found: {retained_comp_file}")
    
    # Create a dictionary for this subject with the required information
    subject_dict = {
        "Subject_id": subject_id,
        "rej_channels": ";".join(rej_channels),
        "rej_channels_reason": ";".join(rej_channels_reason),
        "rej_components": ";".join(rej_components),
        "rej_components_reason": ";".join(rej_components_reason),
        "retained_components": ";".join(retained_components),
        "retained_components_reason": ";".join(retained_components_reason)
    }
    
    # Append the subject dict to the consolidated list
    consolidated_data.append(subject_dict)

# Create a DataFrame from the consolidated data and save as a CSV file
df_consolidated = pd.DataFrame(consolidated_data)
output_csv = join(ica_dir, "summary_ica.csv")
df_consolidated.to_csv(output_csv, index=False)
print(f"Consolidated CSV file saved to: {output_csv}")

# Save the subjects_channel_reject dictionary to a text file
output_txt = join(ica_dir, "subjects_channel_reject.txt")
with open(output_txt, "w") as f:
    f.write(str(subjects_channel_reject))
print(f"Subjects channel reject dictionary saved to: {output_txt}")


Consolidated CSV file saved to: C:\Users\Radovan\OneDrive\Radboud\Studentships\Jordy Thielen\root\data\analysis\ICA\summary_ica.csv
Subjects channel reject dictionary saved to: C:\Users\Radovan\OneDrive\Radboud\Studentships\Jordy Thielen\root\data\analysis\ICA\subjects_channel_reject.txt
